### Install

In [ ]:
!pip list

In [ ]:
# !pip install pytz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Check GPU utilization

In [2]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
# !pip install psutil # already installed
# !pip install humanize

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [3]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed

gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  
printm()

Gen RAM Free: 12.9 GB  | Proc size: 156.0 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


### Imort module

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
# from scipy.misc import imsave
import imageio
import os
import shutil
from PIL import Image
import random
import sys

# time session
import time
from pytz import timezone
from datetime import datetime

### Import .layers

In [ ]:
# import tensorflow as tf

def lrelu(x, leak=0.2, name="lrelu", alt_relu_impl=False):

    with tf.variable_scope(name):
        if alt_relu_impl:
            f1 = 0.5 * (1 + leak)
            f2 = 0.5 * (1 - leak)
            # lrelu = 1/2 * (1 + leak) * x + 1/2 * (1 - leak) * |x|
            return f1 * x + f2 * abs(x)
        else:
            return tf.maximum(x, leak*x)

def instance_norm(x):

    with tf.variable_scope("instance_norm"):
        epsilon = 1e-5
        mean, var = tf.nn.moments(x, [1, 2], keep_dims=True)
        scale = tf.get_variable('scale',[x.get_shape()[-1]], 
            initializer=tf.truncated_normal_initializer(mean=1.0, stddev=0.02))
        offset = tf.get_variable('offset',[x.get_shape()[-1]],initializer=tf.constant_initializer(0.0))
        out = scale*tf.div(x-mean, tf.sqrt(var+epsilon)) + offset

        return out


def general_conv2d(inputconv, o_d=64, f_h=7, f_w=7, s_h=1, s_w=1, stddev=0.02, padding="VALID", name="conv2d", do_norm=True, do_relu=True, relufactor=0):
    with tf.variable_scope(name):
        
        conv = tf.contrib.layers.conv2d(inputconv, o_d, f_w, s_w, padding, activation_fn=None, weights_initializer=tf.truncated_normal_initializer(stddev=stddev),biases_initializer=tf.constant_initializer(0.0))
        if do_norm:
            conv = instance_norm(conv)
            # conv = tf.contrib.layers.batch_norm(conv, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, scope="batch_norm")
            
        if do_relu:
            if(relufactor == 0):
                conv = tf.nn.relu(conv,"relu")
            else:
                conv = lrelu(conv, relufactor, "lrelu")

        return conv



def general_deconv2d(inputconv, outshape, o_d=64, f_h=7, f_w=7, s_h=1, s_w=1, stddev=0.02, padding="VALID", name="deconv2d", do_norm=True, do_relu=True, relufactor=0):
    with tf.variable_scope(name):

        conv = tf.contrib.layers.conv2d_transpose(inputconv, o_d, [f_h, f_w], [s_h, s_w], padding, activation_fn=None, weights_initializer=tf.truncated_normal_initializer(stddev=stddev),biases_initializer=tf.constant_initializer(0.0))
        
        if do_norm:
            conv = instance_norm(conv)
            # conv = tf.contrib.layers.batch_norm(conv, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, scope="batch_norm")
            
        if do_relu:
            if(relufactor == 0):
                conv = tf.nn.relu(conv,"relu")
            else:
                conv = lrelu(conv, relufactor, "lrelu")

        return conv

### Import .model

In [ ]:
# import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# import numpy as np
# from scipy.misc import imsave
# import os
# import shutil
# from PIL import Image
# import time
# import random


# from layers import *

img_height = 256
img_width = 256
img_layer = 3
img_size = img_height * img_width


batch_size = 1
pool_size = 50
ngf = 32
ndf = 64





def build_resnet_block(inputres, dim, name="resnet"):
    
    with tf.variable_scope(name):

        out_res = tf.pad(inputres, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")
        out_res = general_conv2d(out_res, dim, 3, 3, 1, 1, 0.02, "VALID","c1")
        out_res = tf.pad(out_res, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")
        out_res = general_conv2d(out_res, dim, 3, 3, 1, 1, 0.02, "VALID","c2",do_relu=False)
        
        return tf.nn.relu(out_res + inputres)


def build_generator_resnet_6blocks(inputgen, name="generator"):
    with tf.variable_scope(name):
        f = 7
        ks = 3
        
        pad_input = tf.pad(inputgen,[[0, 0], [ks, ks], [ks, ks], [0, 0]], "REFLECT")
        o_c1 = general_conv2d(pad_input, ngf, f, f, 1, 1, 0.02,name="c1")
        o_c2 = general_conv2d(o_c1, ngf*2, ks, ks, 2, 2, 0.02,"SAME","c2")
        o_c3 = general_conv2d(o_c2, ngf*4, ks, ks, 2, 2, 0.02,"SAME","c3")

        o_r1 = build_resnet_block(o_c3, ngf*4, "r1")
        o_r2 = build_resnet_block(o_r1, ngf*4, "r2")
        o_r3 = build_resnet_block(o_r2, ngf*4, "r3")
        o_r4 = build_resnet_block(o_r3, ngf*4, "r4")
        o_r5 = build_resnet_block(o_r4, ngf*4, "r5")
        o_r6 = build_resnet_block(o_r5, ngf*4, "r6")

        o_c4 = general_deconv2d(o_r6, [batch_size,64,64,ngf*2], ngf*2, ks, ks, 2, 2, 0.02,"SAME","c4")
        o_c5 = general_deconv2d(o_c4, [batch_size,128,128,ngf], ngf, ks, ks, 2, 2, 0.02,"SAME","c5")
        o_c5_pad = tf.pad(o_c5,[[0, 0], [ks, ks], [ks, ks], [0, 0]], "REFLECT")
        o_c6 = general_conv2d(o_c5_pad, img_layer, f, f, 1, 1, 0.02,"VALID","c6",do_relu=False)

        # Adding the tanh layer

        out_gen = tf.nn.tanh(o_c6,"t1")


        return out_gen

def build_generator_resnet_9blocks(inputgen, name="generator"):
    with tf.variable_scope(name):
        f = 7
        ks = 3
        
        pad_input = tf.pad(inputgen,[[0, 0], [ks, ks], [ks, ks], [0, 0]], "REFLECT")
        o_c1 = general_conv2d(pad_input, ngf, f, f, 1, 1, 0.02,name="c1")
        o_c2 = general_conv2d(o_c1, ngf*2, ks, ks, 2, 2, 0.02,"SAME","c2")
        o_c3 = general_conv2d(o_c2, ngf*4, ks, ks, 2, 2, 0.02,"SAME","c3")

        o_r1 = build_resnet_block(o_c3, ngf*4, "r1")
        o_r2 = build_resnet_block(o_r1, ngf*4, "r2")
        o_r3 = build_resnet_block(o_r2, ngf*4, "r3")
        o_r4 = build_resnet_block(o_r3, ngf*4, "r4")
        o_r5 = build_resnet_block(o_r4, ngf*4, "r5")
        o_r6 = build_resnet_block(o_r5, ngf*4, "r6")
        o_r7 = build_resnet_block(o_r6, ngf*4, "r7")
        o_r8 = build_resnet_block(o_r7, ngf*4, "r8")
        o_r9 = build_resnet_block(o_r8, ngf*4, "r9")

        o_c4 = general_deconv2d(o_r9, [batch_size,128,128,ngf*2], ngf*2, ks, ks, 2, 2, 0.02,"SAME","c4")
        o_c5 = general_deconv2d(o_c4, [batch_size,256,256,ngf], ngf, ks, ks, 2, 2, 0.02,"SAME","c5")
        o_c6 = general_conv2d(o_c5, img_layer, f, f, 1, 1, 0.02,"SAME","c6",do_relu=False)

        # Adding the tanh layer

        out_gen = tf.nn.tanh(o_c6,"t1")


        return out_gen


def build_gen_discriminator(inputdisc, name="discriminator"):

    with tf.variable_scope(name):
        f = 4

        o_c1 = general_conv2d(inputdisc, ndf, f, f, 2, 2, 0.02, "SAME", "c1", do_norm=False, relufactor=0.2)
        o_c2 = general_conv2d(o_c1, ndf*2, f, f, 2, 2, 0.02, "SAME", "c2", relufactor=0.2)
        o_c3 = general_conv2d(o_c2, ndf*4, f, f, 2, 2, 0.02, "SAME", "c3", relufactor=0.2)
        o_c4 = general_conv2d(o_c3, ndf*8, f, f, 1, 1, 0.02, "SAME", "c4",relufactor=0.2)
        o_c5 = general_conv2d(o_c4, 1, f, f, 1, 1, 0.02, "SAME", "c5",do_norm=False,do_relu=False)

        return o_c5


def patch_discriminator(inputdisc, name="discriminator"):

    with tf.variable_scope(name):
        f= 4

        patch_input = tf.random_crop(inputdisc,[1,70,70,3])
        o_c1 = general_conv2d(patch_input, ndf, f, f, 2, 2, 0.02, "SAME", "c1", do_norm="False", relufactor=0.2)
        o_c2 = general_conv2d(o_c1, ndf*2, f, f, 2, 2, 0.02, "SAME", "c2", relufactor=0.2)
        o_c3 = general_conv2d(o_c2, ndf*4, f, f, 2, 2, 0.02, "SAME", "c3", relufactor=0.2)
        o_c4 = general_conv2d(o_c3, ndf*8, f, f, 2, 2, 0.02, "SAME", "c4", relufactor=0.2)
        o_c5 = general_conv2d(o_c4, 1, f, f, 1, 1, 0.02, "SAME", "c5",do_norm=False,do_relu=False)

        return o_c5

### Import .main

**Should be executed only for ONCE per Session**

- check `output_path` and `check_dir` for your own directory

- training options

    - to_train 
    
    - to_test
    
    - to_restore 
    
> if you want to continue to train previous checkpoints, both `to_train` and `to_restore` must be `True`

In [7]:
# import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# import numpy as np
# from scipy.misc import imsave
# import os
# import shutil
# from PIL import Image
# import time
# import random
# import sys

# from layers import *
# from model import *

img_height = 256 # change image size to your custom dataset's image size
img_width = 256
img_layer = 3
img_size = img_height * img_width

# check train / test / restore

to_train = True
to_test = False
to_restore = False

#------------------

output_path = "/content/drive/My Drive/Colab Notebooks/zebra/output"
check_dir = "/content/drive/My Drive/Colab Notebooks/zebra/output/checkpoints/"


temp_check = 0



max_epoch = 1
max_images = 500 # training atleast 500 images

h1_size = 150
h2_size = 300
z_size = 100
batch_size = 1
pool_size = 50
sample_size = 10
save_training_images = True
ngf = 32 # number of filters for generator (for each layer)
ndf = 64 # number of filters for discriminator

class CycleGAN():

    def input_setup(self):

        ''' 
        This function basically setup variables for taking image input.
        filenames_A/filenames_B -> takes the list of all training images
        self.image_A/self.image_B -> Input image with each values ranging from [-1,1]
        '''

        filenames_A = tf.train.match_filenames_once("/content/drive/My Drive/Colab Notebooks/zebra/horse2zebra/trainA/*.jpg")    
        self.queue_length_A = tf.size(filenames_A)
        filenames_B = tf.train.match_filenames_once("/content/drive/My Drive/Colab Notebooks/zebra/horse2zebra/trainB/*.jpg")    
        self.queue_length_B = tf.size(filenames_B)
        
        filename_queue_A = tf.train.string_input_producer(filenames_A)
        filename_queue_B = tf.train.string_input_producer(filenames_B)

        image_reader = tf.WholeFileReader()
        _, image_file_A = image_reader.read(filename_queue_A)
        _, image_file_B = image_reader.read(filename_queue_B)

        self.image_A = tf.subtract(tf.div(tf.image.resize_images(tf.image.decode_jpeg(image_file_A),[256,256]),127.5),1)
        self.image_B = tf.subtract(tf.div(tf.image.resize_images(tf.image.decode_jpeg(image_file_B),[256,256]),127.5),1)

    

    def input_read(self, sess):


        '''
        It reads the input into from the image folder.
        self.fake_images_A/self.fake_images_B -> List of generated images used for calculation of loss function of Discriminator
        self.A_input/self.B_input -> Stores all the training images in python list
        '''

        # Loading images into the tensors
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)

        num_files_A = sess.run(self.queue_length_A)
        num_files_B = sess.run(self.queue_length_B)

        self.fake_images_A = np.zeros((pool_size,1,img_height, img_width, img_layer))
        self.fake_images_B = np.zeros((pool_size,1,img_height, img_width, img_layer))


        self.A_input = np.zeros((max_images, batch_size, img_height, img_width, img_layer))
        self.B_input = np.zeros((max_images, batch_size, img_height, img_width, img_layer))

        for i in range(max_images): 
            image_tensor = sess.run(self.image_A)
            if(image_tensor.size == img_size*batch_size*img_layer):
                self.A_input[i] = image_tensor.reshape((batch_size,img_height, img_width, img_layer))

        for i in range(max_images):
            image_tensor = sess.run(self.image_B)
            if(image_tensor.size == img_size*batch_size*img_layer):
                self.B_input[i] = image_tensor.reshape((batch_size,img_height, img_width, img_layer))


        coord.request_stop()
        coord.join(threads)




    def model_setup(self):

        ''' This function sets up the model to train
        self.input_A/self.input_B -> Set of training images.
        self.fake_A/self.fake_B -> Generated images by corresponding generator of input_A and input_B
        self.lr -> Learning rate variable
        self.cyc_A/ self.cyc_B -> Images generated after feeding self.fake_A/self.fake_B to corresponding generator. This is use to calcualte cyclic loss
        '''

        self.input_A = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_layer], name="input_A")
        self.input_B = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_layer], name="input_B")
        
        self.fake_pool_A = tf.placeholder(tf.float32, [None, img_width, img_height, img_layer], name="fake_pool_A")
        self.fake_pool_B = tf.placeholder(tf.float32, [None, img_width, img_height, img_layer], name="fake_pool_B")

        self.global_step = tf.Variable(0, name="global_step", trainable=False)

        self.num_fake_inputs = 0

        self.lr = tf.placeholder(tf.float32, shape=[], name="lr")

        with tf.variable_scope("Model") as scope:
            self.fake_B = build_generator_resnet_9blocks(self.input_A, name="g_A")
            self.fake_A = build_generator_resnet_9blocks(self.input_B, name="g_B")
            self.rec_A = build_gen_discriminator(self.input_A, "d_A")
            self.rec_B = build_gen_discriminator(self.input_B, "d_B")

            scope.reuse_variables()

            self.fake_rec_A = build_gen_discriminator(self.fake_A, "d_A")
            self.fake_rec_B = build_gen_discriminator(self.fake_B, "d_B")
            self.cyc_A = build_generator_resnet_9blocks(self.fake_B, "g_B")
            self.cyc_B = build_generator_resnet_9blocks(self.fake_A, "g_A")

            scope.reuse_variables()

            self.fake_pool_rec_A = build_gen_discriminator(self.fake_pool_A, "d_A")
            self.fake_pool_rec_B = build_gen_discriminator(self.fake_pool_B, "d_B")

    def loss_calc(self):

        ''' In this function we are defining the variables for loss calcultions and traning model
        d_loss_A/d_loss_B -> loss for discriminator A/B
        g_loss_A/g_loss_B -> loss for generator A/B
        *_trainer -> Variaous trainer for above loss functions
        *_summ -> Summary variables for above loss functions'''

        cyc_loss = tf.reduce_mean(tf.abs(self.input_A-self.cyc_A)) + tf.reduce_mean(tf.abs(self.input_B-self.cyc_B))
        
        disc_loss_A = tf.reduce_mean(tf.squared_difference(self.fake_rec_A,1))
        disc_loss_B = tf.reduce_mean(tf.squared_difference(self.fake_rec_B,1))
        
        g_loss_A = cyc_loss*10 + disc_loss_B
        g_loss_B = cyc_loss*10 + disc_loss_A

        d_loss_A = (tf.reduce_mean(tf.square(self.fake_pool_rec_A)) + tf.reduce_mean(tf.squared_difference(self.rec_A,1))) / 2.0
        d_loss_B = (tf.reduce_mean(tf.square(self.fake_pool_rec_B)) + tf.reduce_mean(tf.squared_difference(self.rec_B,1))) / 2.0

        
        optimizer = tf.train.AdamOptimizer(self.lr, beta1=0.5)

        self.model_vars = tf.trainable_variables()

        d_A_vars = [var for var in self.model_vars if 'd_A' in var.name]
        g_A_vars = [var for var in self.model_vars if 'g_A' in var.name]
        d_B_vars = [var for var in self.model_vars if 'd_B' in var.name]
        g_B_vars = [var for var in self.model_vars if 'g_B' in var.name]
        
        self.d_A_trainer = optimizer.minimize(d_loss_A, var_list=d_A_vars)
        self.d_B_trainer = optimizer.minimize(d_loss_B, var_list=d_B_vars)
        self.g_A_trainer = optimizer.minimize(g_loss_A, var_list=g_A_vars)
        self.g_B_trainer = optimizer.minimize(g_loss_B, var_list=g_B_vars)

        for var in self.model_vars: print(var.name)

        #Summary variables for tensorboard

        self.g_A_loss_summ = tf.summary.scalar("g_A_loss", g_loss_A)
        self.g_B_loss_summ = tf.summary.scalar("g_B_loss", g_loss_B)
        self.d_A_loss_summ = tf.summary.scalar("d_A_loss", d_loss_A)
        self.d_B_loss_summ = tf.summary.scalar("d_B_loss", d_loss_B)

    def save_training_images(self, sess, epoch):

        if not os.path.exists("/content/drive/My Drive/Colab Notebooks/zebra/output/images/train/"):
            os.makedirs("/content/drive/My Drive/Colab Notebooks/zebra/output/images/train/")

        for i in range(0,11):
            fake_A_temp, fake_B_temp, cyc_A_temp, cyc_B_temp = sess.run([self.fake_A, self.fake_B, self.cyc_A, self.cyc_B],feed_dict={self.input_A:self.A_input[i], self.input_B:self.B_input[i]})
            
            if i == 10:
                imageio.imwrite("/content/drive/My Drive/Colab Notebooks/zebra/output/images/train/"+ str(epoch) + "_" + str(i)+"_fakeB.jpg",((fake_A_temp[0]+1)*127.5).astype(np.uint8))
                imageio.imwrite("/content/drive/My Drive/Colab Notebooks/zebra/output/images/train/"+ str(epoch) + "_" + str(i)+"_fakeA.jpg",((fake_B_temp[0]+1)*127.5).astype(np.uint8))
                imageio.imwrite("/content/drive/My Drive/Colab Notebooks/zebra/output/images/train/"+ str(epoch) + "_" + str(i)+"_cycA.jpg",((cyc_A_temp[0]+1)*127.5).astype(np.uint8))
                imageio.imwrite("/content/drive/My Drive/Colab Notebooks/zebra/output/images/train/"+ str(epoch) + "_" + str(i)+"_cycB.jpg",((cyc_B_temp[0]+1)*127.5).astype(np.uint8))
                imageio.imwrite("/content/drive/My Drive/Colab Notebooks/zebra/output/images/train/"+ str(epoch) + "_" + str(i)+"_inputA.jpg",((self.A_input[i][0]+1)*127.5).astype(np.uint8))
                imageio.imwrite("/content/drive/My Drive/Colab Notebooks/zebra/output/images/train/"+ str(epoch) + "_" + str(i)+"_inputB.jpg",((self.B_input[i][0]+1)*127.5).astype(np.uint8))

    def fake_image_pool(self, num_fakes, fake, fake_pool):
        ''' This function saves the generated image to corresponding pool of images.
        In starting. It keeps on feeling the pool till it is full and then randomly selects an
        already stored image and replace it with new one.'''

        if(num_fakes < pool_size):
            fake_pool[num_fakes] = fake
            return fake
        else :
            p = random.random()
            if p > 0.5:
                random_id = random.randint(0,pool_size-1)
                temp = fake_pool[random_id]
                fake_pool[random_id] = fake
                return temp
            else :
                return fake


    def train(self):


        ''' Training Function '''


        # Load Dataset from the dataset folder
        self.input_setup()  

        #Build the network
        self.model_setup()

        #Loss function calculations
        self.loss_calc()
      
        # Initializing the global variables
        init = [tf.global_variables_initializer(), tf.local_variables_initializer()]
        saver = tf.train.Saver()     

        with tf.Session() as sess:
            sess.run(init)

            #Read input to nd array
            self.input_read(sess)
            
            #-------------------------------------------------------
            
            #Restore the model to run the model from last checkpoint
            if to_restore:
                chkpt_fname = tf.train.latest_checkpoint(check_dir)
                
#                 print(chkpt_fname) # /content/drive/My Drive/Colab Notebooks/zebra/output/checkpoints/cyclegan-99
                
#                 print(str(chkpt_fname).split('/')) # ['', 'content', 'drive', 'My Drive', 'Colab Notebooks', 'zebra', 'output', 'checkpoints', 'cyclegan-99']
                stored_name = str(chkpt_fname).split('/')[8]
#                 print(stored_name.split('-'))
                stored_epoch = int(stored_name.split('-')[1])
                
                # assign previous value
                self.global_step.load(stored_epoch + 1, sess)
                
                saver.restore(sess, chkpt_fname)
                print("Lastest checkpoint has been restored.")
                
            #-------------------------------------------------------

            writer = tf.summary.FileWriter("/content/drive/My Drive/Colab Notebooks/zebra/output/2")

            if not os.path.exists(check_dir):
                os.makedirs(check_dir)
            
            start_epoch = sess.run(self.global_step) # tf.Variable.load(value, session)
            
            # Training Loop
            for epoch in range(start_epoch, start_epoch + 50):                
                print ("In the epoch ", epoch)
                saver.save(sess,os.path.join(check_dir,"cyclegan"),global_step=epoch)

                # Dealing with the learning rate as per the epoch number
                if(epoch < 100) :
                    curr_lr = 0.0002
                else:
                    curr_lr = 0.0002 - 0.0002*(epoch-100)/100

                if(save_training_images):
                    self.save_training_images(sess, epoch)

                # sys.exit()

                for ptr in range(0,max_images):
                    print("In the iteration ",ptr)
#                     print("Starting", time.time()*1000.0)
                    print("Starting", '@', datetime.now(timezone('Asia/Seoul')))

                    # Optimizing the G_A network

                    _, fake_B_temp, summary_str = sess.run([self.g_A_trainer, self.fake_B, self.g_A_loss_summ],feed_dict={self.input_A:self.A_input[ptr], self.input_B:self.B_input[ptr], self.lr:curr_lr})
                    
                    writer.add_summary(summary_str, epoch*max_images + ptr)                    
                    fake_B_temp1 = self.fake_image_pool(self.num_fake_inputs, fake_B_temp, self.fake_images_B)
                    
                    # Optimizing the D_B network
                    _, summary_str = sess.run([self.d_B_trainer, self.d_B_loss_summ],feed_dict={self.input_A:self.A_input[ptr], self.input_B:self.B_input[ptr], self.lr:curr_lr, self.fake_pool_B:fake_B_temp1})
                    writer.add_summary(summary_str, epoch*max_images + ptr)
                    
                    
                    # Optimizing the G_B network
                    _, fake_A_temp, summary_str = sess.run([self.g_B_trainer, self.fake_A, self.g_B_loss_summ],feed_dict={self.input_A:self.A_input[ptr], self.input_B:self.B_input[ptr], self.lr:curr_lr})

                    writer.add_summary(summary_str, epoch*max_images + ptr)
                    
                    
                    fake_A_temp1 = self.fake_image_pool(self.num_fake_inputs, fake_A_temp, self.fake_images_A)

                    # Optimizing the D_A network
                    _, summary_str = sess.run([self.d_A_trainer, self.d_A_loss_summ],feed_dict={self.input_A:self.A_input[ptr], self.input_B:self.B_input[ptr], self.lr:curr_lr, self.fake_pool_A:fake_A_temp1})

                    writer.add_summary(summary_str, epoch*max_images + ptr)
                    
                    self.num_fake_inputs+=1
            
                        

                sess.run(tf.assign(self.global_step, epoch + 1))

            writer.add_graph(sess.graph)

    def test(self):


        ''' Testing Function'''

        print("Testing the results")

        self.input_setup()

        self.model_setup()
        saver = tf.train.Saver()
        init = [tf.global_variables_initializer(), tf.local_variables_initializer()]

        with tf.Session() as sess:

            sess.run(init)

            self.input_read(sess)

            chkpt_fname = tf.train.latest_checkpoint(check_dir)
            saver.restore(sess, chkpt_fname) 

            if not os.path.exists("/content/drive/My Drive/Colab Notebooks/zebra/output/images/test/"):
                os.makedirs("/content/drive/My Drive/Colab Notebooks/zebra/output/images/test/")            

            for i in range(0,100):
                fake_A_temp, fake_B_temp = sess.run([self.fake_A, self.fake_B],feed_dict={self.input_A:self.A_input[i], self.input_B:self.B_input[i]})
                
                if i%20 == 0:
                    imageio.imwrite("/content/drive/My Drive/Colab Notebooks/zebra/output/images/test/"+str(i)+"_fakeB.jpg",((fake_A_temp[0]+1)*127.5).astype(np.uint8))
                    imageio.imwrite("/content/drive/My Drive/Colab Notebooks/zebra/output/images/test/"+str(i)+"_fakeA.jpg",((fake_B_temp[0]+1)*127.5).astype(np.uint8))
                    imageio.imwrite("/content/drive/My Drive/Colab Notebooks/zebra/output/images/test/"+str(i)+"_inputA.jpg",((self.A_input[i][0]+1)*127.5).astype(np.uint8))
                    imageio.imwrite("/content/drive/My Drive/Colab Notebooks/zebra/output/images/test/"+str(i)+"_inputB.jpg",((self.B_input[i][0]+1)*127.5).astype(np.uint8))


def main():
    
    model = CycleGAN()
    if to_train:
        model.train()
    elif to_test:
        model.test()

if __name__ == '__main__':

    main()

W0729 03:02:33.078073 140457605760896 deprecation.py:323] From <ipython-input-7-87e9cb3b668e>:51: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0729 03:02:33.085470 140457605760896 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/input.py:278: input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0729

Model/g_A/c1/Conv/weights:0
Model/g_A/c1/Conv/biases:0
Model/g_A/c1/instance_norm/scale:0
Model/g_A/c1/instance_norm/offset:0
Model/g_A/c2/Conv/weights:0
Model/g_A/c2/Conv/biases:0
Model/g_A/c2/instance_norm/scale:0
Model/g_A/c2/instance_norm/offset:0
Model/g_A/c3/Conv/weights:0
Model/g_A/c3/Conv/biases:0
Model/g_A/c3/instance_norm/scale:0
Model/g_A/c3/instance_norm/offset:0
Model/g_A/r1/c1/Conv/weights:0
Model/g_A/r1/c1/Conv/biases:0
Model/g_A/r1/c1/instance_norm/scale:0
Model/g_A/r1/c1/instance_norm/offset:0
Model/g_A/r1/c2/Conv/weights:0
Model/g_A/r1/c2/Conv/biases:0
Model/g_A/r1/c2/instance_norm/scale:0
Model/g_A/r1/c2/instance_norm/offset:0
Model/g_A/r2/c1/Conv/weights:0
Model/g_A/r2/c1/Conv/biases:0
Model/g_A/r2/c1/instance_norm/scale:0
Model/g_A/r2/c1/instance_norm/offset:0
Model/g_A/r2/c2/Conv/weights:0
Model/g_A/r2/c2/Conv/biases:0
Model/g_A/r2/c2/instance_norm/scale:0
Model/g_A/r2/c2/instance_norm/offset:0
Model/g_A/r3/c1/Conv/weights:0
Model/g_A/r3/c1/Conv/biases:0
Model/g_A

W0729 03:03:16.743000 140457605760896 deprecation.py:323] From <ipython-input-7-87e9cb3b668e>:74: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0729 03:06:26.127129 140457605760896 deprecation.py:323] From <ipython-input-7-87e9cb3b668e>:259: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
W0729 03:06:26.380313 140457605760896 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Lastest checkpoint has been restored.
In the epoch  175
In the iteration  0
Starting @ 2019-07-29 12:06:41.930419+09:00
In the iteration  1
Starting @ 2019-07-29 12:06:54.861652+09:00
In the iteration  2
Starting @ 2019-07-29 12:06:55.198898+09:00
In the iteration  3
Starting @ 2019-07-29 12:06:55.539857+09:00
In the iteration  4
Starting @ 2019-07-29 12:06:55.887432+09:00
In the iteration  5
Starting @ 2019-07-29 12:06:56.231086+09:00
In the iteration  6
Starting @ 2019-07-29 12:06:56.574487+09:00
In the iteration  7
Starting @ 2019-07-29 12:06:56.922417+09:00
In the iteration  8
Starting @ 2019-07-29 12:06:57.264704+09:00
In the iteration  9
Starting @ 2019-07-29 12:06:57.610279+09:00
In the iteration  10
Starting @ 2019-07-29 12:06:57.955471+09:00
In the iteration  11
Starting @ 2019-07-29 12:06:58.302163+09:00
In the iteration  12
Starting @ 2019-07-29 12:06:58.647727+09:00
In the iteration  13
Starting @ 2019-07-29 12:06:58.996726+09:00
In the iteration  14
Starting @ 2019-07-29 1

W0729 03:22:09.145622 140457605760896 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


In the iteration  0
Starting @ 2019-07-29 12:22:11.471299+09:00
In the iteration  1
Starting @ 2019-07-29 12:22:11.824564+09:00
In the iteration  2
Starting @ 2019-07-29 12:22:12.182099+09:00
In the iteration  3
Starting @ 2019-07-29 12:22:12.542135+09:00
In the iteration  4
Starting @ 2019-07-29 12:22:12.897228+09:00
In the iteration  5
Starting @ 2019-07-29 12:22:13.253642+09:00
In the iteration  6
Starting @ 2019-07-29 12:22:13.615683+09:00
In the iteration  7
Starting @ 2019-07-29 12:22:13.974712+09:00
In the iteration  8
Starting @ 2019-07-29 12:22:14.330915+09:00
In the iteration  9
Starting @ 2019-07-29 12:22:14.692005+09:00
In the iteration  10
Starting @ 2019-07-29 12:22:15.048311+09:00
In the iteration  11
Starting @ 2019-07-29 12:22:15.407408+09:00
In the iteration  12
Starting @ 2019-07-29 12:22:15.766321+09:00
In the iteration  13
Starting @ 2019-07-29 12:22:16.126191+09:00
In the iteration  14
Starting @ 2019-07-29 12:22:16.485426+09:00
In the iteration  15
Starting @ 201

KeyboardInterrupt: ignored